## Configuração inicial

In [1]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender

In [2]:
## Configurações base

wandb_key = "b9d02b504b9d57ae7801f351e79da0162387f010"

config = {
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 12,     # número de usuários para filtragem colaborativa
    "nci" :19,      # número de itens para filtragem colaborativa
    "lenlimit" :8,  # limite de tamanho para a lista filmes assistidos 
    "test_run" : 5, # define a quantidade de recomendações, '0' para todos
    "obs": "testando template de prompt"
}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_2
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = ""
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

Quantidade de Usuários: 943


---------   
## Escolha do modelo

### gemma-3-4b-it

In [3]:
config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

### meta-llama-3.1-8b-instruct

In [ ]:
config.update({
    "model_name" :"meta-llama-3.1-8b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

### llama-3.2-3b-instruct

In [ ]:
config.update({
    "model_name" :"llama-3.2-3b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q8_0",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

## Execução

In [4]:
result_pkl = recommender.recommendation_workflow(config         = config,
                                                 wandb_key      = wandb_key,
                                                 dataset        = dataset,
                                                 prompt_template= prompt_template,
                                                 prompt_format  = prompt_format,
                                                 run_wandb      = False)

Processando:   0%|          | 0/5 [00:00<?, ?it/s]

## Resultados

In [5]:
#arq_name = 'ml_100k-zs-nir-su12-ci19-llama-3.2-3b-instruct.pkl'

with open(f'{result_pkl}', 'rb') as f:
    data = pickle.load(f)

#data

In [6]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            'Recommendations': value.get('recommendations', ''),
            'Hit': value.get('hit', ''),
            'Precision': value.get('precision', ''),
            'Recall': value.get('recall', ''),
            'NDCG': value.get('ndcg', '')
        })

df_results = pd.DataFrame(results)

df_results


,Candidates,Ground Truth,gt_in_candidate_set,Input 3,Predictions 3,Recommendations,Hit,Precision,Recall,NDCG
0,"[Clueless, Mrs. Doubtfire, Batman, Heat, Inter...",Aristocats,no,"\n\n ### CANDIDATE MOVIE SET: Clueless, Mrs...",1. Mrs. Doubtfire\n2. Stand by Me\n3. One F...,,0,0.0,0.0,0.000000
1,"[Lone Star, Courage Under Fire, Sleepers, Ever...",River Wild,yes,"\n\n ### CANDIDATE MOVIE SET: Lone Star, Co...",1. Lone Star\n2. Sleepers\n3. Cape Fear\n4....,,1,0.1,1.0,0.356207
2,"[Titanic, George of the Jungle, Full Monty, En...",Paradise Lost: The Child Murders at Robin Hood...,no,"\n\n ### CANDIDATE MOVIE SET: Titanic, Geor...",1. Ulee's Gold\n2. Deconstructing Harry\n3. ...,,0,0.0,0.0,0.000000
3,"[G.I. Jane, Game, I Know What You Did Last Sum...",Seven (Se7en),no,"\n\n ### CANDIDATE MOVIE SET: G.I. Jane, Ga...",1. Ulee’s Gold\n2. Cop Land\n3. Desperate Meas...,,0,0.0,0.0,0.000000
4,"[Scream, Terminator, Usual Suspects, Ace Ventu...",Beverly Hills Cop III,no,"\n\n ### CANDIDATE MOVIE SET: Scream, Termi...",1. Independence Day (ID4)\n2. The Fifth Elemen...,,0,0.0,0.0,0.000000


In [ ]:
df_results['Predictions 3'][0]